In [7]:
import sys
!conda install --yes --prefix {sys.prefix} biopython

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\qwemb\anaconda3

  added / updated specs:
    - biopython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.78             |   py38he774522_0         2.1 MB
    ------------------------------------------------------------
                                           Total:         2.1 MB

The following NEW packages will be INSTALLED:

  biopython          pkgs/main/win-64::biopython-1.78-py38he774522_0




biopython-1.78       | 2.1 MB    |            |   0% 
biopython-1.78       | 2.1 MB    |            |   1% 
biopython-1.78       | 2.1 MB    | #3         |  13% 
biopython-1.78       | 2.1 MB    | #####2     |  52% 
biopython-1.78       | 2.1 MB    | ########## | 100% 
biopython-1.78       | 2.1 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transactio

# Домашнее задание №4

Это домашнее задание можно выполнять целиком в этом ноутбуке, либо алгоритмы написать в отдельном файле и импортировать сюда, для использования. В папке data лежат два файла islands.fasta и nonIslands.fasta. В них хранятся прочтения из CpG островков и из обычных участков генома соответственно, этими данными нужно будет воспользоваться в первом задании.

## Задача №1 (1)
Определите частоты генерации для каждого из нуклеотидов внутри CpG островков и вне их. Посчитайте так-же частоты для всех упорядоченных пар нуклеотидов и сравните частоту пары CG внутри островков и снаружи. Сделайте вывод. 

In [108]:
from Bio import SeqIO
import numpy as np 
import random


In [14]:
def load(name):
    seqs = []
    for record in SeqIO.parse(name, "fasta"):
        seqs.append(record.seq)
    return seqs

In [15]:
islands = load("data/islands.fasta")
nonislands = load("data/nonIslands.fasta")

In [97]:
def sfreq(seq,normalized = True):
        cnt = [0,0,0,0]
        
        for s in seq:
            if s == "A":
                cnt[0] = cnt[0] + 1
            elif s == "C":
                cnt[1] = cnt[1] + 1
            elif s == "G":
                cnt[2] = cnt[2] + 1
            else: 
                cnt[3] = cnt[3] + 1
                
        if normalized:
            return list(map(lambda x: x/sum(cnt),cnt))
        else:
            return cnt
        
def pfreq(seqs,normalized = True):
    cnt = {f:{s:0 for s in "ACGT"} for f in "ACGT"}
    for seq in seqs:
        for f,s in zip(seq[0:-2],seq[1:-1]):
            cnt[f][s] += 1
        
    if normalized:
        l = sum(map(len,seqs))
        for f in cnt.keys():
            for s in cnt[f].keys():
                cnt[f][s] /= l 
        return cnt
    else: 
        return cnt
        
    
def foreach(seq,what = sfreq):
    seqLen = len(seq)
    # cnt = {l:0 for l in "ACGT"}
    res = []
    
    for s in seq:
        freq = what(s,normalized = False)
        
        res.append({l:i for l,i in zip("ACGT",freq)})
    
    cnt = {s:0 for s in "AGTC"}
    for d in res:
        for k,v in d.items():
            cnt[k] += v
    l = sum(map(len,seq))
    return {k: v/l for k,v in cnt.items()}
        

In [106]:
single_island = foreach(islands,sfreq)
pair_island = pfreq(islands)
single_nonisland = foreach(nonislands,sfreq)
pair_nonisland = pfreq(nonislands)

print("single island data prob = ", single_island)
print("single nonisland data prob = ", single_nonisland)

print("single island data prob = ", pair_island['C']['G'])
print("single island data prob = ", pair_nonisland['C']['G'])

single island data prob =  {'A': 0.20246225382932168, 'G': 0.29827680525164113, 'T': 0.20286433260393874, 'C': 0.29639660831509845}
single nonisland data prob =  {'A': 0.28929861153616176, 'G': 0.1984481597792771, 'T': 0.3161168211758391, 'C': 0.19613640750872205}
single island data prob =  0.058940098468271335
single island data prob =  0.005277951721572745


Видим, что частота CG нуклеотидных пар в последовательностях из CpG островков больше, чем из обычных участков генома.

## Задача №2 (2)
Напишите марковскую модель, которая имеет открытые состояния {A, T, G, C}, и скрытые состояния {+, -}. Когда модель в состоянии **+**, то вероятность генерации некоторого символа нуклеотида соответствует его частоте внутри CpG островков, вычислиному в первом задании, если состояние **-**, то частоте вне островков. Вероятность остаться внутри островка 0.95, а перейти в обычный геном 0.05. Для остальной части генома соответствующие вероятности 0.995 и 0.005. Саму модель можно реализовать в виде итератора, определив метод next, который возвращает пару - состояние и нуклеотид, который в этом состоянии произведен.    
Воспользуйтесь данной моделью для того чтобы сгенерировать набор из 20 последовательностей длинной от 1 000 до 100 000, причем к каждой последовательности должна прилагаться последовательность состояний.

In [344]:
def generator(freq_island,freq_nonisland,switch):
    st = random.choice(["+","-"])
    switch = {"+":switch[0]  , "-":switch[1]}
    states = {"+":freq_island, "-":freq_nonisland}
    
    while True:
        st = random.choices(["+","-"], weights=switch[st])[0]
        ch = random.choices(list(states[st].keys()),weights=list(states[st].values()))[0]
        yield ch,st

def advanced_generator(freq_pair_island,freq_pair_nonisland,switch):
    st = random.choice(["+","-"])
    ch = random.choice(['A','G','T','C'])
    switch = {"+":switch[0]  , "-":switch[1]}
    states = {"+":freq_pair_island, "-":freq_pair_nonisland}
    
    st = random.choices(["+","-"], weights=switch[st])[0]
    ch = random.choices(list(states[st].keys()),weights=list(states[st][ch].values()))[0]
    
    while True:
        st = random.choices(["+","-"], weights=switch[st])[0]
        ch = random.choices(list(states[st].keys()),weights=list(states[st][ch].values()))[0]
        yield ch,st

In [347]:
n = 20
min_len = 1000
max_len = 100000


def test(gen):
    res = []
    for i in range(n):

        i = random.randint(min_len,max_len)
        mseq = []
        mst = []

        for _ in range(i):
            ch, st = next(gen)
            mseq.append(ch)
            mst.append(st)
        res.append([mseq,mst])
    
    return res

In [348]:
switch = [[0.95,0.05],[0.005,0.995]]
#gen = advanced_generator(single_island,single_nonisland,switch)

gen1 = generator(single_island,single_nonisland,switch)
gen2 = advanced_generator(pair_island,pair_nonisland,switch)

res1 = test(gen1)
res2 = test(gen2)
       


In [356]:
seqs = []
tr = list(map(lambda x: seqs.append("".join(x[0])),res1))
#print(seqs[0])

## Задача №3 (4)
Напишите алгоритм Витерби для восстановления последовательности скрытых состояний марковской модели из второго задаания. Воспользуйтесь им, воссстановив состояния тех последовательностей, которые вы получили во втором задании и посчитайте TP, TN, FP, FN по количеству правильно или ошибочно предсказанных позиций из CpG остравков. 

## Задача №4 (4)
Напишите алгоритм вперед назад для модели из второго задания. Пользуясь этим алгоритмом найдите вероятности того, что модель находилась в состоянии **+** для каждой позиции строк из второго задания. Устанавливая различные пороговые значения, определите позиции соответствующие CpG островкам и посчитайте TP. Постройте график зависимости TP от выбранного порогового значения. Есть ли пороговые значения при которых TP больше чем в задании №3?